~~~
 -               : rownr in the table starting at 0
 - Bib           : Assigned race number based on qualifying time. "F" + number could appear for female elites.
 - Name          : Name of runner (Last, First)
 - Age           : Age on race day
 - M/F           : Runner's gender
 - City          : Runner's city of residence
 - State         : Runner's state of residence (if applicable).
 - Country       : Runner's country of residence
 - Citizen       : Runner's nationality (optional)
 - 5K            : Runner's time at 5k
 - 10K           : Runner's time at 10k
 - 15K           : Runner's time at 15k
 - 20K           : Runner's time at 20k
 - Half          : Runner's time at halfway point
 - 25K           : Runner's time at 25k
 - 30K           : Runner's time at 30k
 - 35K           : Runner's time at 35k
 - 40K           : Runner's time at 40k
 - Pace          : Runner's overall minute per mile pace
 - Proj Time     : Runner's projected time during the race (empty)
 - Official Time : Runner's official finishing time
 - Overall       : Runner's overall raking
 - Gender        : Runner's ranking in their gender
 - Division      : Runner's ranking in their age division
 ~~~

- 

In [1]:
# 0 - setup

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import pandas as pd
pd.set_option('display.max_columns', 30)
import duckdb
from plotnine import *

# define a sqldf function that uses duckdb to execute sql select queries
sqldf = lambda q: duckdb.query(q).to_df()

In [2]:
# 1 - get the data

dat = pd.read_csv('./dat/marathon_results_2017.csv',index_col=False)
print(dat.shape)
dat.head(3)

(26410, 25)


,Unnamed: 0,Bib,Name,Age,M/F,City,State,Country,Citizen,Unnamed: 9,5K,10K,15K,20K,Half,25K,30K,35K,40K,Pace,Proj Time,Official Time,Overall,Gender,Division
0,0,11,"Kirui, Geoffrey",24,M,Keringet,NaN,KEN,NaN,NaN,0:15:25,0:30:28,0:45:44,1:01:15,1:04:35,1:16:59,1:33:01,1:48:19,2:02:53,0:04:57,-,2:09:37,1,1,1
1,1,17,"Rupp, Galen",30,M,Portland,OR,USA,NaN,NaN,0:15:24,0:30:27,0:45:44,1:01:15,1:04:35,1:16:59,1:33:01,1:48:19,2:03:14,0:04:58,-,2:09:58,2,2,2
2,2,23,"Osako, Suguru",25,M,Machida-City,NaN,JPN,NaN,NaN,0:15:25,0:30:29,0:45:44,1:01:16,1:04:36,1:17:00,1:33:01,1:48:31,2:03:38,0:04:59,-,2:10:28,3,3,3


In [3]:
# Rename needed col names to something palatable 
betterColNames = ['runID', 'bib', 'name', 'age', 'sex', 'city', 'state', 'country',
       'citizen', 'dummy', '5K', '10K', '15K', '20K', 'halfTime', '25K',
       '30K', '35K', '40K', 'pace', 'dummy2', 'endTime', 'overAllRank',
       'sexeRank', 'divRank']
dat.columns = betterColNames
dat.head(1)

,runID,bib,name,age,sex,city,state,country,citizen,dummy,5K,10K,15K,20K,halfTime,25K,30K,35K,40K,pace,dummy2,endTime,overAllRank,sexeRank,divRank
0,0,11,"Kirui, Geoffrey",24,M,Keringet,NaN,KEN,NaN,NaN,0:15:25,0:30:28,0:45:44,1:01:15,1:04:35,1:16:59,1:33:01,1:48:19,2:02:53,0:04:57,-,2:09:37,1,1,1


all about [pandas read_htmo](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html)  
all about [divisions in the Boston Marathon](https://www.runnerstribe.com/latest-news/want-to-run-the-boston-marathon-here-is-everything-you-need-to-know/)

In [4]:
url = 'https://www.runnerstribe.com/latest-news/want-to-run-the-boston-marathon-here-is-everything-you-need-to-know/'
pd.read_html(url, match='Age Group')[0]

,Age Group,MEN,WOMEN
0,18-34,3hrs 00min 00sec,3hrs 30min 00sec
1,35-39,3hrs 05min 00sec,3hrs 35min 00sec
2,40-44,3hrs 10min 00sec,3hrs 40min 00sec
3,45-49,3hrs 20min 00sec,3hrs 50min 00sec
4,50-54,3hrs 25min 00sec,3hrs 55min 00sec
5,55-59,3hrs 35min 00sec,4hrs 05min 00sec
6,60-64,3hrs 50min 00sec,4hrs 20min 00sec
7,65-69,4hrs 05min 00sec,4hrs 35min 00sec
8,70-74,4hrs 20min 00sec,4hrs 50min 00sec
9,75-79,4hrs 35min 00sec,5hrs 05min 00sec


In [5]:
def get_age_group(xxx):
    if   xxx < 35 : return 'AG_1'
    elif xxx < 40 : return 'AG_2'
    elif xxx < 45 : return 'AG_3'
    elif xxx < 50 : return 'AG_4'
    elif xxx < 55 : return 'AG_5'
    elif xxx < 60 : return 'AG_6'
    elif xxx < 65 : return 'AG_7'
    elif xxx < 70 : return 'AG_8'
    elif xxx < 75 : return 'AG_9'
    elif xxx < 80 : return 'AG_A'
    else          : return 'AG_B'

dat['ageGrp'] = dat['age'].apply(get_age_group)

In [6]:
dat.columns


Index(['runID', 'bib', 'name', 'age', 'sex', 'city', 'state', 'country',
       'citizen', 'dummy', '5K', '10K', '15K', '20K', 'halfTime', '25K', '30K',
       '35K', '40K', 'pace', 'dummy2', 'endTime', 'overAllRank', 'sexeRank',
       'divRank', 'ageGrp'],
      dtype='object')

In [7]:
# rename the columns
better_col_names = [
'runID', 'bib', 'name', 'age', 'sex', 'city', 'state', 'country',
'citizen', 'dummy', 'K05', 'K10', 'K15', 'K20', 'halfTime', 'K25', 'K30',
'K35', 'K40', 'pace', 'dummy2', 'endTime', 'overAllRank', 'sexeRank',
'divRank', 'ageGrp']

dat.columns = better_col_names

In [8]:
# remove the dummy's + reorder 
dat = sqldf("""
SELECT 
    runID, bib, name, age, ageGrp, sex, 
    concat(sex, '_', ageGrp) AS division,
    city, state, country, citizen, 
    K05, K10, K15, K20, halfTime, K25, K30, K35, K40, endTime, pace, 
    overAllRank, sexeRank, divRank
FROM dat
ORDER BY runID
""")
dat.head()

,runID,bib,name,age,ageGrp,sex,division,city,state,country,citizen,K05,K10,K15,K20,halfTime,K25,K30,K35,K40,endTime,pace,overAllRank,sexeRank,divRank
0,0,11,"Kirui, Geoffrey",24,AG_1,M,M_AG_1,Keringet,NaN,KEN,NaN,0:15:25,0:30:28,0:45:44,1:01:15,1:04:35,1:16:59,1:33:01,1:48:19,2:02:53,2:09:37,0:04:57,1,1,1
1,1,17,"Rupp, Galen",30,AG_1,M,M_AG_1,Portland,OR,USA,NaN,0:15:24,0:30:27,0:45:44,1:01:15,1:04:35,1:16:59,1:33:01,1:48:19,2:03:14,2:09:58,0:04:58,2,2,2
2,2,23,"Osako, Suguru",25,AG_1,M,M_AG_1,Machida-City,NaN,JPN,NaN,0:15:25,0:30:29,0:45:44,1:01:16,1:04:36,1:17:00,1:33:01,1:48:31,2:03:38,2:10:28,0:04:59,3,3,3
3,3,21,"Biwott, Shadrack",32,AG_1,M,M_AG_1,Mammoth Lakes,CA,USA,NaN,0:15:25,0:30:29,0:45:44,1:01:19,1:04:45,1:17:00,1:33:01,1:48:58,2:04:35,2:12:08,0:05:03,4,4,4
4,4,9,"Chebet, Wilson",31,AG_1,M,M_AG_1,Marakwet,NaN,KEN,NaN,0:15:25,0:30:28,0:45:44,1:01:15,1:04:35,1:16:59,1:33:01,1:48:41,2:05:00,2:12:35,0:05:04,5,5,5
